In [ ]:
#! pip install fairlearn
#! pip install lightgbm
#! pip install optuna

In [4]:
from sklearn.metrics import f1_score, confusion_matrix, make_scorer, accuracy_score, recall_score
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from fairlearn.datasets import fetch_adult
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

import numpy as np
import optuna
import tqdm as notebook_tqdm
from metrics import (
    equality_opportunity_difference,
    predictive_equality_difference,
    metrics
)

In [5]:
data = fetch_adult(as_frame=True)
X_raw = data.data
y = (data.target == ">50K") * 1
A = X_raw["sex"]

numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            LGBMClassifier(n_jobs=-1),
        ),
    ]
)

/home/azucena/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [6]:
results = []


In [ ]:

directions = ['minimize', 'maximize']
metric_scorer_decorated =  metrics(recall_score, predictive_equality_difference, sensitive_col = 'sex')
for sim in [1]:
    print(sim)
    def objective(trial):

        (X_train, X_test, y_train, y_test, A_train, A_test) = train_test_split(
        X_raw, y, A, test_size=0.8, random_state=sim, stratify=y
        )

        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)
        A_train = A_train.reset_index(drop=True)
        A_test = A_test.reset_index(drop=True)

        classifier_name = trial.suggest_categorical("classifier", ["logit", "RF", 'GBM','LGBM'])

        if classifier_name == "logit":        
            params = {
                "penalty" : trial.suggest_categorical('logit_penalty', ['l1','l2']),
                "c" : trial.suggest_float('logit_c', 0.001, 10),
                "solver" : 'saga'
                }
        
        elif classifier_name =="RF":
            params = {
                'n_estimators': trial.suggest_int("rf_n_estimators", 100, 1000),
                'criterion': trial.suggest_categorical("rf_criterion", ['gini', 'entropy']),
                'max_depth': trial.suggest_int("rf_max_depth", 1, 4),
                'min_samples_split': trial.suggest_float("rf_min_samples_split", 0.01, 1),
                }

        elif classifier_name =="LGBM":
            params = {
                'n_estimators': trial.suggest_int("n_estimators", 20, 10000),
                'num_leaves': trial.suggest_int("num_leaves", 10, 1000),
                'max_depth': trial.suggest_int("max_depth", 2, 20),
                'min_child_samples': trial.suggest_int("min_child_samples", 5, 300),
                'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-2),
                'boosting_type': trial.suggest_categorical("boosting_type", ['goss', 'gbdt'])
                }
        
         elif classifier_name =="GBM":
            params = {
                'n_estimators': trial.suggest_int("gbm_n_estimators", 100, 1000),
                'gbm_criterion': trial.suggest_categorical("gbm_criterion", ['mse', 'friedman_mse']),
                'max_depth': trial.suggest_int("gbm_max_depth", 1, 4),
                'min_samples_split': trial.suggest_int("gbm_min_child_samples", 5, 300),
                }
        
        else:
            None
        


        
        
        pipeline['classifier'].set_params(**params)

        scores = cross_validate(
                pipeline, 
                X_train,
                y_train, 
                cv=5,
                scoring = metric_scorer_decorated,
                return_train_score=True)

        fair_metric = scores['test_fairness'].mean()
        model_metric = scores['test_model'].mean()

        return fair_metric, model_metric
    
    
    
    study = optuna.create_study(
        directions = directions, 
        pruner = optuna.pruners.SuccessiveHalvingPruner(), 
        sampler = optuna.samplers.TPESampler() 
        )
    study.optimize(objective, n_trials=100)
    print("Number of finished trials: ", len(study.trials))
    results.append(study)

In [ ]:
import dill
file_name = 'recall-fpr-lgbm-motpe-succesivehalving-100trials-4sim.pkl'
#f1-eod-lgbm-succesivehalving-30trails.pkl
with open(file_name, 'wb') as file:
    dill.dump(results, file)
    print(f'Object successfully saved to "{file_name}"')

In [ ]:
optuna.visualization.plot_pareto_front(study, target_names=["FLOPS", "accuracy"])